# Lake Depth RF to Raster Data

In [1]:
# System modules
import os
import time
from pprint import pprint

# Anything numeric related
import numpy as np
import matplotlib.pyplot as plt

# Anything geospatial related
import rasterio as rio

# Anything GPU/ML related
import cupy as cp
import cuml
import cudf
from models import custom_RF as crf

### Helper functions

In [2]:
def list_files(directory, extension):
    """ Helper function to read in files to list"""
    return list((f for f in os.listdir(directory) if f.endswith('.' + extension)))

In [3]:
def timer(start, end):
    """ Helper function for timing things """
    print (end - start)
    return (end - start)

### Useful file paths

In [4]:
TIF_PATH = '/att/nobackup/maronne/lake/forCaleb/'
TIF_FILES = list(list_files(TIF_PATH, 'tif'))
CURRENT_TIF = TIF_FILES[7]
pprint(TIF_FILES)

['LC8_075011_20160706_stack_clip.tif',
 'LC8_075011_20170810_stack_clip.tif',
 'LC8_078011_20160711_stack_clip.tif',
 'LC8_077010_20180710_stack_clip.tif',
 'LC8_080010_20170712_stack_clip.tif',
 'LC8_075011_20170709_stack_clip.tif',
 'LC8_078011_20180701_stack_clip.tif',
 'LC8_080010_20170728_stack_clip.tif',
 'LC8_075011_20160807_stack_clip.tif',
 'LC8_077011_20160720_stack_clip.tif',
 'LC8_075011_20160722_stack_clip.tif',
 'LC8_079011_20180724_stack_clip.tif',
 'LC8_079010_20180724_stack_clip.tif',
 'LC8_077010_20160720_stack_clip.tif',
 'LC8_080010_20160709_stack_clip.tif',
 'LC8_076011_20160830_stack_clip.tif',
 'LC8_080011_20160725_stack_clip.tif',
 'LC8_077011_20160704_stack_clip.tif',
 'LC8_078011_20160812_stack_clip.tif']


## Using rasterio for all raster-related things

The with statement is nice because it automatically closes the file after getting the necessary data

In [5]:
with rio.open(TIF_PATH+CURRENT_TIF) as raster_img:
    """ 
        The with statement is nice becuase it auto
        closes all rasterio tifs opened once below
        code is executed
    """
    n_cols = raster_img.width
    n_rows = raster_img.height
    n_bands = raster_img.count
    gt = raster_img.transform
    crs = raster_img.crs
    ndval = raster_img.nodata
    img_properties = (n_cols, n_rows, n_bands, gt, crs)
    
    st_0 = time.time()
    # Create numpy array to mimic tif
    img_nd = np.zeros((n_rows, n_cols, n_bands), np.float32) 
    for b in range(n_bands):
        print(" - reading in band #", b)
        st_1 = time.time()
        img_nd[:, :, b] = raster_img.read(b+1) # Populate it with band pixel vals
        et_1 = time.time()
        timer(st_1, et_1)
    et_0 = time.time()
    timer(st_0, et_0)

 - reading in band # 0
9.142392635345459
 - reading in band # 1
0.35376477241516113
 - reading in band # 2
0.3525512218475342
 - reading in band # 3
0.35111212730407715
 - reading in band # 4
0.35419392585754395
 - reading in band # 5
0.3577840328216553
 - reading in band # 6
0.35756444931030273
 - reading in band # 7
0.3577606678009033
 - reading in band # 8
0.3587477207183838
 - reading in band # 9
0.35716700553894043
 - reading in band # 10
0.35738062858581543
 - reading in band # 11
0.35813117027282715
 - reading in band # 12
0.3584327697753906
 - reading in band # 13
0.3571295738220215
 - reading in band # 14
0.35422492027282715
 - reading in band # 15
0.35535383224487305
 - reading in band # 16
0.3992147445678711
 - reading in band # 17
0.3936634063720703
 - reading in band # 18
0.4149761199951172
 - reading in band # 19
0.41744208335876465
 - reading in band # 20
0.41695213317871094
 - reading in band # 21
0.3583974838256836
 - reading in band # 22
0.3574349880218506
 - reading 

In [6]:
from pprint import pprint
print('\n Image properties \n')
pprint(img_properties)
print("\n Image as ndarray \n")
pprint(img_nd)
print('\n Sample of bands, one row \n')
pprint(img_nd[5350, 5880, ])
# What one 'row' of bands look like
print(img_nd.shape)
print(np.unique(img_nd))


 Image properties 

(5881,
 5351,
 35,
 Affine(30.008648019487495, 0.0, 340610.828997643,
       0.0, -30.0040523395777, 7915630.04458002),
 CRS.from_epsg(32605))

 Image as ndarray 

array([[[ 2.3300e+02,  1.7400e+02,  3.9000e+01, ...,  2.6000e+01,
          5.0000e+02,  2.5000e+02],
        [ 2.3500e+02,  1.6900e+02,  4.5000e+01, ...,  1.1100e+02,
          5.0000e+03,  1.6670e+03],
        [ 2.3400e+02,  1.7100e+02,  4.5000e+01, ...,  1.3300e+02,
          3.2767e+04,  2.0000e+03],
        ...,
        [-9.9990e+03, -9.9990e+03, -9.9990e+03, ..., -9.9990e+03,
         -9.9990e+03, -9.9990e+03],
        [-9.9990e+03, -9.9990e+03, -9.9990e+03, ..., -9.9990e+03,
         -9.9990e+03, -9.9990e+03],
        [-9.9990e+03, -9.9990e+03, -9.9990e+03, ..., -9.9990e+03,
         -9.9990e+03, -9.9990e+03]],

       [[ 2.2800e+02,  1.7300e+02,  4.0000e+01, ...,  1.2500e+02,
          1.0000e+03,  1.6670e+03],
        [ 2.3100e+02,  1.6600e+02,  4.5000e+01, ...,  1.1100e+02,
          5.0000e+03

In [7]:
new_shape = (img_nd.shape[0] * img_nd.shape[1], img_nd.shape[2])
img_nd_array = img_nd[:, :, :img_nd.shape[2]].reshape(new_shape)

In [8]:
print(img_nd_array.shape)
pprint(img_nd[5350, 5880, 0])
pprint(img_nd_array[(5350*5880), 0])

(31469231, 35)
-9999.0
95.0


### Convert our data to GPU-accelerated data
np array -> cupy array -> cudf Dataframe

In [9]:
gpu_img = cp.asarray(img_nd_array)

In [10]:
cdf_raster = cudf.DataFrame(gpu_img)

In [11]:
n_rows_raster, n_cols_raster = cdf_raster.shape
pprint(cdf_raster)
print(n_rows_raster)
print(n_cols_raster)

              0       1       2       3       4       5       6       7   \
0          233.0   174.0    39.0     2.0     4.0     1.0  1339.0  5974.0   
1          235.0   169.0    45.0     1.0     3.0     5.0  1391.0  5222.0   
2          234.0   171.0    45.0     0.0     3.0     6.0  1368.0  5200.0   
3          231.0   169.0    47.0     0.0     4.0     3.0  1367.0  4915.0   
4          231.0   170.0    45.0     1.0     2.0     2.0  1359.0  5133.0   
...          ...     ...     ...     ...     ...     ...     ...     ...   
31469226 -9999.0 -9999.0 -9999.0 -9999.0 -9999.0 -9999.0 -9999.0 -9999.0   
31469227 -9999.0 -9999.0 -9999.0 -9999.0 -9999.0 -9999.0 -9999.0 -9999.0   
31469228 -9999.0 -9999.0 -9999.0 -9999.0 -9999.0 -9999.0 -9999.0 -9999.0   
31469229 -9999.0 -9999.0 -9999.0 -9999.0 -9999.0 -9999.0 -9999.0 -9999.0   
31469230 -9999.0 -9999.0 -9999.0 -9999.0 -9999.0 -9999.0 -9999.0 -9999.0   

               8        9   ...      25      26      27      28       29  \
0         3

#### Load up our perviously trained random forest model

In [12]:
model_load = crf.load_model('best_test_03.sav')

## Predictions from raster data
- We need to split it up due to GPU memory limitations

In [23]:
index_0 = n_rows_raster//2
index_1 = index_0 * 2 
index_3 = n_rows_raster % 2 # Any indeces left out?
print(index_0)
print(index_1)

15734615
31469230


Predict first half then second half, no need for any third prediction since total is even

In [24]:
predictions_1 = model_load.model.predict(cdf_raster[:index_0]) # Predict first half
predictions_2 = model_load.model.predict(cdf_raster[index_0:]) # Predict other hals
# No indeces left out so no need for any more predictions

In [25]:
cp.unique(predictions_1) # Looks like this is ndvals

array([ 0.36128747,  0.37853485,  0.38806912, ..., 12.548393  ,
       13.236887  , 13.264563  ], dtype=float32)

In [26]:
cp.unique(predictions_2)

array([ 0.2872215,  0.2952346,  0.3019142, ..., 16.339315 , 16.651136 ,
       17.037361 ], dtype=float32)

Put our predictions back by together by rows

In [27]:
concat_predictions = cudf.concat([predictions_1, predictions_2], axis=0)
print(concat_predictions.shape)

(31469231,)


In [28]:
array_predictions = concat_predictions.to_array()

In [29]:
array_predictions = array_predictions.reshape(img_nd[:, :, 0].shape).astype(np.float32)

In [ ]:
pprint(array_predictions)

In [ ]:
print(array_predictions.shape)

Make sure we put our ndvals back

In [ ]:
array_predictions[img_nd[:, :, 0] == -9999.0] = ndval

In [ ]:
pprint(array_predictions)

In [ ]:
file_name_no_extension = CURRENT_TIF.split('.', 1)[0]
print(file_name_no_extension)
file_name_predicted = file_name_no_extension + '_predicted_0.tif'
print(file_name_predicted)

In [ ]:
with rio.open(file_name_predicted, 
                   'w', 
                   driver='GTiff', 
                   height = array_predictions.shape[0],
                   width = array_predictions.shape[1],
                   count = 1,
                   dtype = array_predictions.dtype,
                   crs = crs,
                   transform = gt) as prediction_raster:
    prediction_raster.write(array_predictions, 1)

In [ ]:
with rio.open(file_name_predicted) as raster_img_predicted:
    """ 
        The with statement is nice becuase it auto
        closes all rasterio tifs opened once below
        code is executed
    """
    band1 = raster_img_predicted.read(1)

In [ ]:
import matplotlib.pyplot as plt
plt.figure(figsize=(12, 12))
plt.imshow(band1)

In [ ]:
with rio.open(TIF_PATH + CURRENT_TIF) as raster_img_original:
    """ 
        The with statement is nice becuase it auto
        closes all rasterio tifs opened once below
        code is executed
    """
    band1 = raster_img_original.read(1)
    band2 = raster_img_original.read(2)
    band3 = raster_img_original.read(3)

In [ ]:
rgb_original_tif = np.dstack((band1, band2, band3))
plt.figure(figsize=(12, 12))
plt.imshow(rgb_original_tif)